## Criando solução com RAG

In [15]:
# Separar o documento em chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Embeddings
from langchain_openai import OpenAIEmbeddings
# LLM
from langchain_openai import ChatOpenAI
# Banco de dados vetorial
from langchain_community.vectorstores import Chroma
# Carregar arquivos PDF
from langchain_community.document_loaders import PyPDFLoader
# Prompt
from langchain.chains.question_answering import load_qa_chain

In [16]:
import os

In [17]:
## Chave API

Load dos modelos (Embeddings e LLM)

In [18]:
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens = 200)

Carregar o arquivo (PDF)

In [19]:
pdf_link = "datasets/remuneracao_docentes_UF_2020.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

Separar em Chunks (separar o conteúdo do arquivo em pequenas partes)

In [20]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 4000,
  chunk_overlap = 20,
  length_function = len,
  add_start_index = True,
)

chunks = text_splitter.split_documents(pages)

Salvar no Vector DB

In [ ]:
db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")
db.persist()